# Understanding Technical Indicators

This notebook walks you through what technical indicators are and how to calculate them using Python.

## What are Technical Indicators?

**Technical indicators** are mathematical calculations based on a stock's price, volume, or open interest. Traders use them to:

1. **Identify trends** - Is the stock going up, down, or sideways?
2. **Find entry/exit points** - When should I buy or sell?
3. **Measure momentum** - How strong is the price movement?
4. **Spot reversals** - Is the trend about to change?

Think of them like **weather forecasting tools**:
- Raw stock prices = temperature readings every minute
- Technical indicators = trends like "7-day average temperature" or "humidity rising fast"

They don't predict the future perfectly, but they help you see patterns in the noise.

## Setup: Import Libraries

We'll use:
- `yfinance` - Download real stock data from Yahoo Finance
- `pandas` - Manipulate time-series data
- `matplotlib` - Create charts
- `numpy` - Mathematical operations

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Make charts look nice
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 7)

## 1. Simplest Example: 20-Day Simple Moving Average (SMA)

The **Simple Moving Average** is the most basic indicator. It's just the average price over the last N days.

**Formula**: SMA = (Price₁ + Price₂ + ... + PriceN) / N

Let's calculate a 20-day SMA for Apple stock:

In [ ]:
# Download 6 months of Apple stock data
ticker = "AAPL"
data = yf.download(ticker, period="6mo", interval="1d")

# Calculate 20-day Simple Moving Average
data['SMA_20'] = data['Close'].rolling(window=20).mean()

# Show the last 5 rows
print(data[['Close', 'SMA_20']].tail())

In [ ]:
# Visualize: Price vs 20-Day SMA
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Daily Close Price', linewidth=1.5, alpha=0.7)
plt.plot(data.index, data['SMA_20'], label='20-Day SMA', linewidth=2, color='orange')
plt.title(f'{ticker} Price vs 20-Day Simple Moving Average', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()

### What Happened?

1. **`data['Close'].rolling(window=20)`** - Creates a sliding window of 20 days
2. **`.mean()`** - Calculates the average of each window
3. **The orange line is smoother** than the blue line because it averages out daily fluctuations

**Why is this useful?**
- Raw prices jump around (noise)
- The SMA smooths out the noise to show the underlying trend
- If price is above SMA → uptrend
- If price is below SMA → downtrend

**Real-World Analogy**: 
- Daily prices = your mood every hour (fluctuates)
- 20-day SMA = your average mood this month (smoother picture)

## 🧪 Experiment: Change the Window Size

Try different window sizes:
- **Short window (20 days)** - Reacts quickly to price changes, more volatile
- **Medium window (50 days)** - Balanced
- **Long window (200 days)** - Slow to react, very smooth

Let's compare all three:

In [ ]:
# Download 1 year of data for better visualization
data = yf.download(ticker, period="1y", interval="1d")

# Calculate multiple SMAs
data['SMA_20'] = data['Close'].rolling(window=20).mean()
data['SMA_50'] = data['Close'].rolling(window=50).mean()
data['SMA_200'] = data['Close'].rolling(window=200).mean()

# Plot all together
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price', linewidth=1.5, alpha=0.5)
plt.plot(data.index, data['SMA_20'], label='20-Day SMA', linewidth=2)
plt.plot(data.index, data['SMA_50'], label='50-Day SMA', linewidth=2)
plt.plot(data.index, data['SMA_200'], label='200-Day SMA', linewidth=2)
plt.title(f'{ticker} - Comparing Different SMA Windows', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()

### What Do You Notice?

- **20-Day SMA** (blue) hugs the price closely - good for short-term trading
- **50-Day SMA** (orange) smoother - medium-term trend
- **200-Day SMA** (green) very smooth - long-term trend ("Is this a bull or bear market?")

**Trading Signal**: When the 50-day SMA crosses above the 200-day SMA, it's called a **"Golden Cross"** (bullish signal)!

## 2. Build Up: Exponential Moving Average (EMA)

The **Exponential Moving Average** is like SMA, but it gives more weight to recent prices.

**Why?** Because yesterday's price is more relevant than the price 20 days ago.

**Formula**: EMA = (Price_today × K) + (EMA_yesterday × (1 - K))
- Where K = 2 / (window + 1)

In [ ]:
# Calculate 20-day EMA
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()

# Compare SMA vs EMA
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price', linewidth=1.5, alpha=0.5)
plt.plot(data.index, data['SMA_20'], label='20-Day SMA', linewidth=2)
plt.plot(data.index, data['EMA_20'], label='20-Day EMA', linewidth=2, linestyle='--')
plt.title(f'{ticker} - SMA vs EMA Comparison', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()

### What's the Difference?

- **EMA** (dashed line) reacts faster to price changes
- **SMA** (solid line) is slower and smoother
- Day traders prefer EMA for quicker signals
- Long-term investors prefer SMA for stability

## 3. Build Up: Relative Strength Index (RSI)

**RSI** measures if a stock is **overbought** (too high, might drop) or **oversold** (too low, might rise).

**Scale**: 0 to 100
- **RSI > 70** → Overbought (sell signal)
- **RSI < 30** → Oversold (buy signal)
- **RSI = 50** → Neutral

**Formula**:
1. Calculate average gains and losses over 14 days
2. RS = Average Gain / Average Loss
3. RSI = 100 - (100 / (1 + RS))

In [ ]:
# Calculate RSI manually
def calculate_rsi(data, window=14):
    # Calculate price changes
    delta = data['Close'].diff()
    
    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    
    # Calculate RS and RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

data['RSI'] = calculate_rsi(data)

# Visualize RSI
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Top chart: Price
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2)
ax1.set_title(f'{ticker} Price and RSI', fontsize=16)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.legend(fontsize=12)
ax1.grid(alpha=0.3)

# Bottom chart: RSI
ax2.plot(data.index, data['RSI'], label='RSI', linewidth=2, color='purple')
ax2.axhline(70, color='red', linestyle='--', label='Overbought (70)')
ax2.axhline(30, color='green', linestyle='--', label='Oversold (30)')
ax2.fill_between(data.index, 70, 100, alpha=0.2, color='red')
ax2.fill_between(data.index, 0, 30, alpha=0.2, color='green')
ax2.set_ylabel('RSI', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.legend(fontsize=12)
ax2.grid(alpha=0.3)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

### What Happened?

1. **Red shaded area (RSI > 70)** - Stock is overbought, might drop soon
2. **Green shaded area (RSI < 30)** - Stock is oversold, might rise soon
3. **Purple line oscillates** between 0 and 100

**How to Use RSI**:
- When RSI crosses below 30 → potential buy signal
- When RSI crosses above 70 → potential sell signal
- Works best in sideways markets (not strong trends)

## 4. Build Up: MACD (Moving Average Convergence Divergence)

**MACD** shows the relationship between two moving averages. It's used to spot:
- Trend changes
- Momentum shifts
- Buy/sell signals

**Components**:
1. **MACD Line** = 12-day EMA - 26-day EMA
2. **Signal Line** = 9-day EMA of MACD Line
3. **Histogram** = MACD Line - Signal Line

**Trading Signal**:
- MACD crosses above Signal → Buy
- MACD crosses below Signal → Sell

In [ ]:
# Calculate MACD
def calculate_macd(data, fast=12, slow=26, signal=9):
    # Calculate EMAs
    ema_fast = data['Close'].ewm(span=fast, adjust=False).mean()
    ema_slow = data['Close'].ewm(span=slow, adjust=False).mean()
    
    # MACD Line = Fast EMA - Slow EMA
    macd_line = ema_fast - ema_slow
    
    # Signal Line = 9-day EMA of MACD
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    
    # Histogram = MACD - Signal
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram

data['MACD'], data['Signal'], data['Histogram'] = calculate_macd(data)

# Visualize MACD
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Top chart: Price
ax1.plot(data.index, data['Close'], label='Close Price', linewidth=2)
ax1.set_title(f'{ticker} Price and MACD', fontsize=16)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.legend(fontsize=12)
ax1.grid(alpha=0.3)

# Bottom chart: MACD
ax2.plot(data.index, data['MACD'], label='MACD Line', linewidth=2, color='blue')
ax2.plot(data.index, data['Signal'], label='Signal Line', linewidth=2, color='red')
ax2.bar(data.index, data['Histogram'], label='Histogram', color='gray', alpha=0.3)
ax2.axhline(0, color='black', linewidth=0.5)
ax2.set_ylabel('MACD', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.legend(fontsize=12)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### What Happened?

1. **Blue line (MACD)** - The difference between fast and slow EMAs
2. **Red line (Signal)** - A smoothed version of the MACD
3. **Gray bars (Histogram)** - The distance between MACD and Signal

**How to Read It**:
- When blue crosses above red → Bullish signal (buy)
- When blue crosses below red → Bearish signal (sell)
- Histogram growing taller → Momentum increasing
- Histogram shrinking → Momentum decreasing

## 5. Build Up: Bollinger Bands

**Bollinger Bands** show the price range where a stock typically trades.

**Components**:
1. **Middle Band** = 20-day SMA
2. **Upper Band** = Middle Band + (2 × Standard Deviation)
3. **Lower Band** = Middle Band - (2 × Standard Deviation)

**What It Tells You**:
- 95% of price action happens between the bands
- Price touching upper band → might be too high
- Price touching lower band → might be too low
- Bands squeeze together → volatility about to increase
- Bands widen → high volatility

In [ ]:
# Calculate Bollinger Bands
def calculate_bollinger_bands(data, window=20, num_std=2):
    # Middle band = SMA
    middle_band = data['Close'].rolling(window=window).mean()
    
    # Standard deviation
    std = data['Close'].rolling(window=window).std()
    
    # Upper and lower bands
    upper_band = middle_band + (std * num_std)
    lower_band = middle_band - (std * num_std)
    
    return upper_band, middle_band, lower_band

data['BB_Upper'], data['BB_Middle'], data['BB_Lower'] = calculate_bollinger_bands(data)

# Visualize Bollinger Bands
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Close'], label='Close Price', linewidth=2, color='black')
plt.plot(data.index, data['BB_Upper'], label='Upper Band', linewidth=1.5, color='red', linestyle='--')
plt.plot(data.index, data['BB_Middle'], label='Middle Band (SMA)', linewidth=1.5, color='blue')
plt.plot(data.index, data['BB_Lower'], label='Lower Band', linewidth=1.5, color='green', linestyle='--')
plt.fill_between(data.index, data['BB_Upper'], data['BB_Lower'], alpha=0.1, color='gray')
plt.title(f'{ticker} - Bollinger Bands', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.show()

### What Happened?

1. **Gray shaded area** - The "normal" trading range
2. **Black line (Price)** - Bounces between the bands most of the time
3. **When price touches upper red band** - Potentially overvalued
4. **When price touches lower green band** - Potentially undervalued

**Trading Strategy**:
- Buy when price touches the lower band
- Sell when price touches the upper band
- Works best in sideways markets (not strong trends)

## 6. Visualization: All Indicators Together

Let's create a comprehensive dashboard with all indicators:

In [ ]:
# Create a 4-panel chart
fig, axes = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

# Panel 1: Price with Bollinger Bands
axes[0].plot(data.index, data['Close'], label='Close', linewidth=2, color='black')
axes[0].plot(data.index, data['BB_Upper'], label='Upper BB', linewidth=1, color='red', linestyle='--', alpha=0.7)
axes[0].plot(data.index, data['BB_Middle'], label='SMA 20', linewidth=1, color='blue', alpha=0.7)
axes[0].plot(data.index, data['BB_Lower'], label='Lower BB', linewidth=1, color='green', linestyle='--', alpha=0.7)
axes[0].fill_between(data.index, data['BB_Upper'], data['BB_Lower'], alpha=0.1, color='gray')
axes[0].set_title(f'{ticker} - Technical Indicators Dashboard', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Price ($)', fontsize=11)
axes[0].legend(loc='upper left', fontsize=10)
axes[0].grid(alpha=0.3)

# Panel 2: Moving Averages
axes[1].plot(data.index, data['Close'], label='Close', linewidth=1.5, alpha=0.5, color='gray')
axes[1].plot(data.index, data['SMA_20'], label='SMA 20', linewidth=2)
axes[1].plot(data.index, data['SMA_50'], label='SMA 50', linewidth=2)
axes[1].plot(data.index, data['EMA_20'], label='EMA 20', linewidth=2, linestyle='--')
axes[1].set_ylabel('Price ($)', fontsize=11)
axes[1].legend(loc='upper left', fontsize=10)
axes[1].grid(alpha=0.3)

# Panel 3: RSI
axes[2].plot(data.index, data['RSI'], label='RSI', linewidth=2, color='purple')
axes[2].axhline(70, color='red', linestyle='--', linewidth=1, alpha=0.7)
axes[2].axhline(30, color='green', linestyle='--', linewidth=1, alpha=0.7)
axes[2].fill_between(data.index, 70, 100, alpha=0.2, color='red')
axes[2].fill_between(data.index, 0, 30, alpha=0.2, color='green')
axes[2].set_ylabel('RSI', fontsize=11)
axes[2].set_ylim(0, 100)
axes[2].legend(loc='upper left', fontsize=10)
axes[2].grid(alpha=0.3)

# Panel 4: MACD
axes[3].plot(data.index, data['MACD'], label='MACD', linewidth=2, color='blue')
axes[3].plot(data.index, data['Signal'], label='Signal', linewidth=2, color='red')
axes[3].bar(data.index, data['Histogram'], label='Histogram', color='gray', alpha=0.3)
axes[3].axhline(0, color='black', linewidth=0.5)
axes[3].set_ylabel('MACD', fontsize=11)
axes[3].set_xlabel('Date', fontsize=12)
axes[3].legend(loc='upper left', fontsize=10)
axes[3].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### How to Read This Dashboard:

1. **Panel 1 (Bollinger Bands)** - Is price in normal range or at extremes?
2. **Panel 2 (Moving Averages)** - What's the trend direction?
3. **Panel 3 (RSI)** - Is the stock overbought or oversold?
4. **Panel 4 (MACD)** - Is momentum increasing or decreasing?

**Strong Buy Signal** = All of these:
- Price near lower Bollinger Band
- RSI < 30 (oversold)
- MACD crossing above Signal
- Price above 200-day SMA (long-term uptrend)

**Strong Sell Signal** = All of these:
- Price near upper Bollinger Band
- RSI > 70 (overbought)
- MACD crossing below Signal
- Price below 200-day SMA (long-term downtrend)

## 🏋️ Practice Exercise: Find RSI Buy Signals

Your task: Find all the dates when RSI gave a buy signal (crossed below 30) in the last 6 months.

**Steps**:
1. Download 6 months of data for a stock of your choice
2. Calculate RSI
3. Find all dates where RSI < 30
4. Print those dates with the RSI value and stock price
5. Calculate what the return would be if you bought on each signal and held for 10 days

Try it yourself first, then check the solution below!

In [ ]:
# Your turn: Complete this exercise

# Step 1: Download data
ticker = "TSLA"  # Change to any stock you want
# ... your code here ...

# Step 2: Calculate RSI
# ... your code here ...

# Step 3: Find RSI < 30 dates
# ... your code here ...

# Step 4: Print results
# ... your code here ...

# Step 5: Calculate 10-day returns
# ... your code here ...

## 💡 Solution (Try yourself first!)

In [ ]:
# Step 1: Download data
ticker = "TSLA"
data = yf.download(ticker, period="6mo", interval="1d")

# Step 2: Calculate RSI
data['RSI'] = calculate_rsi(data, window=14)

# Step 3: Find dates where RSI < 30 (oversold)
oversold = data[data['RSI'] < 30].copy()

# Step 4: Print results
print(f"\nRSI Buy Signals for {ticker} (RSI < 30 = Oversold)")
print("=" * 70)

if len(oversold) == 0:
    print(f"No oversold signals found in the last 6 months for {ticker}")
else:
    for date, row in oversold.iterrows():
        print(f"Date: {date.strftime('%Y-%m-%d')} | RSI: {row['RSI']:.2f} | Price: ${row['Close']:.2f}")
    
    # Step 5: Calculate 10-day forward returns
    print("\n" + "=" * 70)
    print("10-Day Forward Returns (if you bought on signal date):")
    print("=" * 70)
    
    for date, row in oversold.iterrows():
        # Get the price 10 days later
        try:
            future_date = date + pd.Timedelta(days=10)
            future_price = data.loc[data.index >= future_date, 'Close'].iloc[0]
            buy_price = row['Close']
            return_pct = ((future_price - buy_price) / buy_price) * 100
            
            print(f"Buy on {date.strftime('%Y-%m-%d')} at ${buy_price:.2f} → "
                  f"Sell 10 days later at ${future_price:.2f} → "
                  f"Return: {return_pct:+.2f}%")
        except IndexError:
            print(f"Buy on {date.strftime('%Y-%m-%d')} at ${buy_price:.2f} → "
                  f"Not enough data to calculate 10-day return")
    
    # Calculate average return
    returns = []
    for date, row in oversold.iterrows():
        try:
            future_date = date + pd.Timedelta(days=10)
            future_price = data.loc[data.index >= future_date, 'Close'].iloc[0]
            return_pct = ((future_price - row['Close']) / row['Close']) * 100
            returns.append(return_pct)
        except IndexError:
            pass
    
    if returns:
        avg_return = np.mean(returns)
        print("\n" + "=" * 70)
        print(f"Average 10-day return from RSI < 30 signals: {avg_return:+.2f}%")
        print("=" * 70)

### What Did We Learn?

1. **Not all signals are profitable** - Technical indicators aren't magic
2. **Context matters** - RSI works better in sideways markets than strong trends
3. **Combine multiple indicators** - Don't rely on just one signal
4. **Backtesting is key** - Always test strategies on historical data first

**Important Disclaimer**: Past performance does not guarantee future results. Always do your own research before trading!

## Key Takeaways

### What We Covered:

1. **Simple Moving Average (SMA)** - Smooths price data to show trends
   - Short window = reactive, Long window = smooth

2. **Exponential Moving Average (EMA)** - Weighted average favoring recent prices
   - Reacts faster than SMA

3. **Relative Strength Index (RSI)** - Measures overbought/oversold conditions
   - RSI > 70 = overbought (sell signal)
   - RSI < 30 = oversold (buy signal)

4. **MACD** - Shows momentum and trend changes
   - MACD crosses above Signal = bullish
   - MACD crosses below Signal = bearish

5. **Bollinger Bands** - Shows volatility and price extremes
   - Price at upper band = potentially overvalued
   - Price at lower band = potentially undervalued

### Best Practices:

1. **Never use one indicator alone** - Combine multiple signals
2. **Backtest your strategies** - Test on historical data first
3. **Understand market context** - Is it trending or sideways?
4. **Use proper risk management** - Don't bet everything on one signal
5. **Keep learning** - Markets change, strategies need adjustment

### Next Steps:

- Try different stocks (tech, finance, energy)
- Experiment with different indicator parameters
- Combine indicators to create trading strategies
- Learn about volume-based indicators (OBV, Volume Profile)
- Study candlestick patterns alongside indicators

**Remember**: Technical indicators are tools, not crystal balls. Use them to inform decisions, not make them automatically!